In [24]:
from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
import os
from sqlalchemy import delete,insert
from sqlalchemy.orm import sessionmaker

import json
import numpy as np
import pandas as pd
from datetime import date,datetime,timedelta

import sqla_schema as sch

import ingest

data_directory = 'C:\\Users\\yoni.browning\\Documents\\DataJoint\\AllenData'
manifest_path = os.path.join(data_directory, "manifest.json")

def get_first_value(value):
    if isinstance(value,str):
        if '[' in value and ']' in value:
            value = np.fromstring(value[1:-1],sep = ',')
            value = value[0]
        else:
            value = float(value)
    elif isinstance(value,np.ndarray) and len(value)==1:
        value = value[0]
    elif isinstance(value,int) or isinstance(value,float):
        value =value
    else:
        value = None
    return value

def query_to_df(Q):
    df = pd.read_sql(Q.statement, Q.session.bind)
    return df

In [3]:
engine = ingest.connect_to_db()
print('Connected to engine')
# Distroy any existing versions of these table
#sch.Base.metadata.drop_all(engine, tables=(sch.StimulusType.__table__,))
#sch.Base.metadata.drop_all(engine, tables=(sch.Stimulus.__table__,))
#print('Killed old tables')

# Generate a new version of these tables
sch.Base.metadata.create_all(engine, tables=(sch.StimulusType.__table__,))
#sch.Base.metadata.create_all(engine, tables=(sch.Stimulus.__table__,))
print('Spawned new tables')

# 
cache = ingest.get_ecephys_cache(manifest = manifest_path)

# Add Stimulus Type arguments. 
# This bit is a little funky, but you only need to do it once.
session = cache.get_session_data(715093703)
print('Grabbed some data')

rf_stim_table = session.stimulus_presentations
stimulus_name_df = \
    pd.DataFrame(data={"stimulus_name": rf_stim_table['stimulus_name'].unique()})
stimulus_name_df.to_sql('stimulus_type', engine, index_label='id', if_exists='append')
print('Added to DB')

Connected to engine
Spawned new tables
Grabbed some data


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "stimulus_name" of relation "stimulus_type" does not exist
LINE 1: INSERT INTO stimulus_type (id, stimulus_name) VALUES (0, 'sp...
                                       ^

[SQL: INSERT INTO stimulus_type (id, stimulus_name) VALUES (%(id)s, %(stimulus_name)s)]
[parameters: ({'id': 0, 'stimulus_name': 'spontaneous'}, {'id': 1, 'stimulus_name': 'gabors'}, {'id': 2, 'stimulus_name': 'flashes'}, {'id': 3, 'stimulus_name': 'drifting_gratings'}, {'id': 4, 'stimulus_name': 'natural_movie_three'}, {'id': 5, 'stimulus_name': 'natural_movie_one'}, {'id': 6, 'stimulus_name': 'static_gratings'}, {'id': 7, 'stimulus_name': 'natural_scenes'})]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [17]:
stimulus_name_df = \
    pd.DataFrame(data={"stimulus_name": rf_stim_table['stimulus_name'].unique()})
stimulus_name_df.columns = stimulus_name_df.columns.str.lower()

stimulus_name_df.to_sql('stimulus_type', engine, index_label='id', if_exists='append')


ProgrammingError: (psycopg2.errors.UndefinedColumn) column "stimulus_name" of relation "stimulus_type" does not exist
LINE 1: INSERT INTO stimulus_type (id, stimulus_name) VALUES (0, 'sp...
                                       ^

[SQL: INSERT INTO stimulus_type (id, stimulus_name) VALUES (%(id)s, %(stimulus_name)s)]
[parameters: ({'id': 0, 'stimulus_name': 'spontaneous'}, {'id': 1, 'stimulus_name': 'gabors'}, {'id': 2, 'stimulus_name': 'flashes'}, {'id': 3, 'stimulus_name': 'drifting_gratings'}, {'id': 4, 'stimulus_name': 'natural_movie_three'}, {'id': 5, 'stimulus_name': 'natural_movie_one'}, {'id': 6, 'stimulus_name': 'static_gratings'}, {'id': 7, 'stimulus_name': 'natural_scenes'})]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [18]:
stimulus_name_df

,stimulus_name
0,spontaneous
1,gabors
2,flashes
3,drifting_gratings
4,natural_movie_three
5,natural_movie_one
6,static_gratings
7,natural_scenes


In [19]:
S = sessionmaker(engine)()
query_to_df(S.query(sch.StimulusType))

ProgrammingError: (psycopg2.errors.UndefinedColumn) column stimulus_type.stimulus_name does not exist
LINE 1: SELECT stimulus_type.id, stimulus_type.stimulus_name 
                                 ^

[SQL: SELECT stimulus_type.id, stimulus_type.stimulus_name 
FROM stimulus_type]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [29]:
insert(sch.StimulusType).values('id' = 0,'stimulus_name'='hello')

SyntaxError: keyword can't be an expression (Temp/ipykernel_16964/59888176.py, line 1)

In [32]:
S.add(sch.StimulusType(stimulus_name = 'hello'))

In [36]:
query_to_df(S.query(sch.StimulusType))

ProgrammingError: (psycopg2.errors.UndefinedColumn) column stimulus_type.stimulus_name does not exist
LINE 1: SELECT stimulus_type.id, stimulus_type.stimulus_name 
                                 ^

[SQL: SELECT stimulus_type.id, stimulus_type.stimulus_name 
FROM stimulus_type]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [39]:
sch.Base.metadata.drop_all(engine, tables=(sch.StimulusType.__table__,))


InternalError: (psycopg2.errors.DependentObjectsStillExist) cannot drop table stimulus_type because other objects depend on it
DETAIL:  constraint stimulus_presentation_stimulus_type_id_fkey on table stimulus_presentation depends on table stimulus_type
HINT:  Use DROP ... CASCADE to drop the dependent objects too.

[SQL: 
DROP TABLE stimulus_type]
(Background on this error at: https://sqlalche.me/e/14/2j85)